In [17]:
import pandas as pd
import numpy as np
from numpy import array
from numpy import mean
from numpy import std
import tensorflow as tf
from tensorflow import keras
from keras import optimizers
from keras.models import Sequential
from keras import callbacks
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed, Conv1D, MaxPooling1D, Flatten, Bidirectional, Input, Flatten, Activation, Reshape, RepeatVector, Concatenate
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt


In [18]:
val_metrics = np.load("val_metric_dict_1.npy", allow_pickle=True)

In [19]:
indexes=[]
for j in range(1, 6):
    for i in range(1,11):
        indexes.append('month '+str(j)+' lookahead ' + str(i))

In [20]:
val_df = pd.DataFrame(val_metrics.reshape(val_metrics.shape[0], val_metrics.shape[2]), columns = ['RMSE', 'MSE', 'MAE', 'MAPE', 'R2'])

In [21]:
val_df.index=indexes[:len(val_df)]

In [22]:
val_df

,RMSE,MSE,MAE,MAPE,R2
month 1 lookahead 1,12.376456,153.176655,7.584862,0.108375,0.732109
month 1 lookahead 2,12.768328,163.030210,7.994822,0.115634,0.714876
month 1 lookahead 3,15.195494,230.903025,9.647211,0.140318,0.596346
month 1 lookahead 4,15.361128,235.964257,9.925636,0.144487,0.587498
month 1 lookahead 5,15.376129,236.425346,9.976249,0.146421,0.586692
month 1 lookahead 6,15.438880,238.359031,10.251598,0.153463,0.583311
month 1 lookahead 7,16.361531,267.699688,10.857685,0.158916,0.532237
month 1 lookahead 8,16.365004,267.813369,10.905205,0.161039,0.532039
month 1 lookahead 9,16.379630,268.292279,10.982768,0.162635,0.531202
month 1 lookahead 10,17.130776,293.463503,11.773480,0.177709,0.487432


In [23]:
val_df.groupby(np.arange(len(val_df))//10).mean()

,RMSE,MSE,MAE,MAPE,R2
0,15.275336,235.512736,9.989952,0.146900,0.588374
1,43.550795,1918.286721,22.908082,0.147781,0.540548
2,38.397212,1477.946397,15.233959,0.193397,0.395016
3,10.994356,122.252973,7.049506,0.178777,0.634658
4,22.195763,506.859130,10.291628,0.190210,0.552142


In [24]:
averaged_months= val_df.iloc[0::10,:].mean().to_frame().transpose()
for i in range(1,10):
    averaged_months = averaged_months.append((val_df.iloc[i::10,:]).mean(), ignore_index=True)

In [25]:
averaged_months

,RMSE,MSE,MAE,MAPE,R2
0,21.207406,578.130162,10.200445,0.130939,0.701420
1,22.557190,647.950256,10.994541,0.145533,0.664453
2,23.772254,710.976318,11.803522,0.152820,0.623763
3,25.753024,808.518360,12.849846,0.170160,0.548519
4,27.160486,914.124645,13.444957,0.172445,0.511093
5,27.142965,916.280536,13.705081,0.176692,0.513502
6,27.755313,939.535100,13.987311,0.185450,0.480864
7,28.380600,996.973184,14.749779,0.197142,0.465103
8,28.380363,996.589459,14.305427,0.185066,0.464833
9,28.717322,1012.637895,14.905345,0.197884,0.447927


RMSE     21.293357
MSE     579.563224
MAE      10.140989
MAPE      0.129148
R2        0.698510
RMSE     21.370282
MSE     579.450893
MAE      10.352163
MAPE      0.135814
R2        0.692253
dtype: float64